# Naver 영화댓글 분류

In [1]:
pip install accelerate -U

   ---------------------------------------- 0.0/309.4 kB ? eta -:--:--
   --------------- ------------------------ 122.9/309.4 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 309.4/309.4 kB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install datasets

  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp311-cp311-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp311-cp311-win_amd64.whl.metadata (32 kB)
   ---------------------------------------- 0.0/542.1 kB ? eta -:--:--
   --------------------------------------- 542.1/542.1 kB 17.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 6.6 MB/s eta 0:00:00
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
   ---------------------------------------- 0.0/370.8 kB ? eta -:--:--
   ---------------------------------------- 370.8/370.8 kB ? eta 0:00:00
   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
   ----- ---------------------------------- 3.4/25.9 MB 71.9 MB/s eta 0:00:01


In [1]:
# pip install accelerate -U
# 설치후 커널 재시작

# Huggingface Dataset 패키지
- 허깅페이스 허브에 공유된 데이터셋을  다운로드해서 전처리 및 관리할 수있도록 돕는 라이브러리. 
- 많은 공개데이터셋을 동일한 인터페이스로 사용할 수있다.
- 설치
    - `pip install datasets`
- https://huggingface.co/datasets
- https://github.com/huggingface/datasets
      
## Huggingface Dataset loading
- datasets 로딩
    - `load_data('dataset name')`
        - huggingface datasets에 등록된 Dataset 이름 넣어 Loading한다.
          
![img](figures/huggingface_dataset.png)

In [2]:
import datasets
datasets.__version__

'2.19.2'

In [3]:
from datasets import load_dataset
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
nsmc = load_dataset("e9t/nsmc")
print(type(nsmc))

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

<class 'datasets.dataset_dict.DatasetDict'>


In [5]:
nsmc

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [14]:
nsmc['train']["document"][:5]  # list
nsmc['train']['label'][:5]
nsmc['train']['id'][:5]

['9976970', '3819312', '10265843', '9045019', '6483659']

In [18]:
# nsmc['train'].to_pandas()

In [20]:
sent_lex = load_dataset('senti-lex/senti_lex', "ko")
sent_lex

Generating train split:   0%|          | 0/2118 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['word', 'sentiment'],
        num_rows: 2118
    })
})

In [23]:
# sent_lex['train']['word']
# sent_lex['train']['sentiment']

In [26]:
## 데이터셋 구성. train: 150,000   test: 50,000
train_X = nsmc['train']['document']
train_y = nsmc['train']['label']
test_X = nsmc['test']['document']
test_y = nsmc['test']['document']

In [27]:
### 데이터 일부만 sampling 
# Dataset
a = nsmc['train'].shuffle().select(range(1000))    # 0 ~ 9999 번째 값만 sub sampling
a

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 1000
})

## 모델, 토크나이저 loading

- 모델 별 Model 클래스를 이용하거나 Auto class를 이용해 모델, 전처리기(tokenizer, ImageProcessor 등)을 로딩한다.
    - Huggingface에 저장된 model name을 입력해서 pretrained 모델을 loading 한다.
    - fine tuning 한 경우 모델 저장 디렉토리 경로를 넣어 pretrained 모델을 loading한다.
- AutoModel은 model name을 주면 그 모델이 학습한 base 모델에 맞는 객체를 생성해서 반환한다.
    - Auto Model은 task 별로 다양한 클래스들이 있다.
        - 클래스 이름 형식: AutoModelFor{Task형식}
        - ex) `AutoModelForObjectDetection`, `AutoModelForSequenceClassification`
    - https://huggingface.co/docs/transformers/model_doc/auto
    - 전처리기(tokenzier)는 사용하려는 모델이 사용한 전처리기를 사용해야 한다.

In [34]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "beomi/kcbert-base"  #backbone 이름.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
# num_labels는 분류할 class의 개수.
## Backbone network는 미리학습된 beomi/kcbert-base 모델을 사용.
## Estimator network는 학습안된 layer를 추가해서 제공.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                                       Param #
BertForSequenceClassification                                --
├─BertModel: 1-1                                             --
│    └─BertEmbeddings: 2-1                                   --
│    │    └─Embedding: 3-1                                   23,040,000
│    │    └─Embedding: 3-2                                   230,400
│    │    └─Embedding: 3-3                                   1,536
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─BertEncoder: 2-2                                      --
│    │    └─ModuleList: 3-6                                  85,054,464
│    └─BertPooler: 2-3                                       --
│    │    └─Linear: 3-7                                      590,592
│    │    └─Tanh: 3-8                                        --
├─Dropout: 1-2                                               --
├─L

In [33]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## pytorch Dataset 생성
모델 입력으로 다음 4개 항목을 dictionary로 묶어서 제공하도록 구현한다.
1. input_ids: 입력 text 토큰을 id로 변환한 값
2. token_type_ids: 문자쌍 구분시 사용. 단일 문장: 0, 문자쌍-첫문장: 0, 두 번째 문장: 1
3. attention_mask: 실제 토큰값과 패딩구분값
4. labels: 정답 class index

1 ~ 3은 위의 train_encoding, test_encoding으로 만듬. labels은 train_data/test_data의 label 키 값 사용

In [39]:
# X (댓글) -> 토큰화
train_encoding = tokenizer(
    train_X, 
    return_tensors='pt', # 토큰화 처리결과들을 torch.Tensor 로 반환.
    padding=True,      # 패딩 방식 - 제일 토큰수가 많은 문장에 맞춘다.
)
test_encoding = tokenizer(
    test_X, 
    return_tensors='pt',
    padding=True    
)

In [45]:
train_encoding.keys()
type(train_encoding['input_ids'])
train_encoding['input_ids'].shape  # [150000:문장수, 142:토큰수]
len(tokenizer)

30000

In [48]:
a = {
    "input_ids":[1, 2, 3, 4, 5],
    "attention_mask":[10, 20, 30, 40, 50],
    "token_type_ids":[100, 200, 300, 400, 500]
    
}
idx = 3
{key: value[3]  for key, value in a.items()}
# {"input_ids":4, "attention_maks":40, "token_type_ids":400}

{'input_ids': 4, 'attention_mask': 40, 'token_type_ids': 400}

In [52]:
###################### Dataset 정의 #################
import torch
from torch.utils.data import Dataset

class NSMCDataset(Dataset):

    def __init__(self, encodings, labels):
        """
        Parameter
            encodings: tokenizer로 encoding 된 댓글.
            labels: 정답 라벨들
        """
        self.encodings = encodings # DatasetDict
        self.labels = labels   # List

    def __getitem__(self, index):
        """
        index 번째의 학습/검증/테스트 데이터를 반환.
        BERT 모델 입력 형식에 맞춰서 반환. bert_model(input_ids, token_type_ids, attention_mask)
        Parameter
            index: int - 반환할 데이터의 index
        Return
            dictionary - input_ids, token_type_ids, attention_mask, label 을 딕셔너리에 묶어서 반환.
        """
        data = {key: value[index]  for key, value in self.encodings.items()}  # label 뺀 dict 
        # data 에 label 추가
        data["labels"] = torch.tensor(self.labels[index], dtype=torch.long)     # label 추가.
        
        return data
    
    def __len__(self):
        return len(self.labels)

In [53]:
train_set = NSMCDataset(train_encoding, train_y)
test_set = NSMCDataset(test_encoding, test_y)
len(train_set), len(test_set)

(150000, 50000)

In [54]:
a = train_set[0]
a

{'input_ids': tensor([    2,  2170,   832,  5045,    17,    17,  7992, 29734,  4040, 10720,
             3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

# 학습
- Transformers는 model 학습을 위해 TrainingArguments, Trainer 클래스를 제공한다.
- TrainingArguments Trainer를 위한 설정을 하는 클래스
- TrainingArguments, Trainer를 이용하면 training option, logging, gradient accumulation, mixed precision등을 쉽게 설정해 학습, 평가를 모두 진행할 수 있다.

In [58]:
1 * 150000 / 128

117187.5

In [60]:
from transformers import TrainingArguments, Trainer

# 총 step:  epoch수 * 데이터수 / batch_size

##  학습(fine tuning) 관련된 설정.
args = TrainingArguments(
    output_dir="models/nsmc", # 학습 도중 모델이 저장될 디렉토리.
    logging_dir = "logging/nsmc", # 학습 도중 생성되는 기록(log)를 저장할 디렉토리.
    num_train_epochs=1,         # 학습 에폭수
    per_device_train_batch_size=128, # batch size (학습)
    per_device_eval_batch_size=128, # batch size (검증, 평가)
    logging_steps=50,    # 몇 step에 한번씩 로그를 저장할지. 
    save_steps=50,        # 몇 step에 한번씩 모델을 저장할지
)

In [61]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [66]:
from datasets import load_metric
# acc_fn = load_metric('accuracy')
acc_fn = load_metric('f1')
acc_fn.compute(references=[0, 1, 1], predictions=[1, 1, 1])

{'f1': 0.8}

In [68]:
def compute_metric(pred):
    """
    모델 학습하는 도중에 예측값과 정답을 받아서 평가점수(accuracy)를 계산. (callback 함수)
    Parameter
        pred: EvalPrediction - 예측값, 정답들을 묶어서 제공.
    Return
        dict : key-평가지표이름, value: 평가점수
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(dim=-1) # 모델 추론값([0일확률, 1일확률])에서 class 추출

    metrics1 = load_metric('accuracy')
    metrics2 = load_metric('f1')
    acc = metrics1(references=labels, predictions=preds)
    f1 = metrics2(references=labels, predictions=preds)
    return {"accuracy":acc, "f1 score": f1}

In [69]:
# 학습
## Trainer 객체 생성
trainer = Trainer(
    model=model, # 학습할 대상 모델
    args=args,       # TrainingArguments
    train_dataset=train_set, # 학습 데이터셋    trainer.train()
    eval_dataset=test_set,   # 검증 데이터셋    trainer.evaluate()
    compute_metrics=compute_metric       #  평가 함수를 등록.
)

In [70]:
# train
trainer.train()

Step,Training Loss


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x00000276F94F8950>>
Traceback (most recent call last):
  File "C:\Classes\DA-35\10_nlp_deeplearning\env\Lib\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1501, in enumerate
    def enumerate():
    
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
trainer.evaluate() # 평가 (eval_dataset)

In [71]:
# 최종 모델 저장 -> fine tuning한 모델, tokenizer
# model.save_pretrained("저장할 디렉토리")
model.save_pretrained("models/my_nsmc")
tokenizer.save_pretrained("models/my_nsmc")

('models/my_nsmc\\tokenizer_config.json',
 'models/my_nsmc\\special_tokens_map.json',
 'models/my_nsmc\\vocab.txt',
 'models/my_nsmc\\added_tokens.json',
 'models/my_nsmc\\tokenizer.json')

## 모델 로드

In [75]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# 파인튜닝한 모델/토크나이저 디렉토리 경로
path = "models/text_cls_model/"
load_tokenzer = AutoTokenizer.from_pretrained(path)
load_model = AutoModelForSequenceClassification.from_pretrained(path)

In [76]:
type(load_tokenzer), type(load_model)

(transformers.models.distilbert.tokenization_distilbert_fast.DistilBertTokenizerFast,
 transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification)

# 추론

In [91]:
# 대상(댓글) 문자열 -> list ->tokenizer 토큰화 -> model 추론 
sentence = ["이걸 영화라고 만든 거냐?", 
            "아무 기대 없이 봤는데 재미있네.", 
            "연기 죽이네.", "그냥 OTT로 볼껄. 극장표값이 아깝다.", "또 보고 싶다. 너무 재미있었다."]
sent_encoding = load_tokenzer(
    sentence, 
    return_tensors="pt",
    padding=True
)
sent_encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [92]:
with torch.no_grad():
    output = load_model(**sent_encoding)

In [93]:
output.logits

tensor([[ 2.6738, -2.2030],
        [-2.2332,  1.8189],
        [ 0.3708, -0.2615],
        [ 2.8151, -2.2008],
        [-2.5266,  2.0443]])

In [94]:
output.logits.softmax(dim=-1)

tensor([[0.9924, 0.0076],
        [0.0171, 0.9829],
        [0.6530, 0.3470],
        [0.9934, 0.0066],
        [0.0102, 0.9898]])

In [95]:
for comment, label in zip(sentence, output.logits.softmax(dim=-1)):
    label_str = "긍정적 댓글" if label.argmax(dim=-1).item() == 1 else "부정적 댓글"
    label_pred = label.max().item()
    print(f"{comment} - {label_str} - {label_pred * 100:.2f}%")

이걸 영화라고 만든 거냐? - 부정적 댓글 - 99.24%
아무 기대 없이 봤는데 재미있네. - 긍정적 댓글 - 98.29%
연기 죽이네. - 부정적 댓글 - 65.30%
그냥 OTT로 볼껄. 극장표값이 아깝다. - 부정적 댓글 - 99.34%
또 보고 싶다. 너무 재미있었다. - 긍정적 댓글 - 98.98%
